In [1]:
import requests
import pandas as pd
from binance.client import Client
from secrets import secrets

auth = requests.auth.HTTPBasicAuth(secrets['REDDIT_API_KEY'], secrets['REDDIT_SECRET_KEY'])
headers = {'User-Agent': 'MyBot/0.0.1'}

data = {'grant_type': 'password', 'username': secrets['REDDIT_USERNAME'], 'password': secrets['REDDIT_PASSWORD']}
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
TOKEN = res.json()['access_token']
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [2]:
BINANCE_API_KEY = secrets['BINANCE_API_KEY']
BINANCE_SECRET_KEY = secrets['BINANCE_SECRET_KEY']
client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)
info = client.get_exchange_info()
coins = [s['baseAsset'] for s in info['symbols']]
coins = list(set(coins))
len(coins)

392

In [86]:
df = pd.DataFrame(columns=['coin', 'mentions'])
df['coin'] = coins
df['mentions'] = 0
df.head()

,coin,mentions
0,LIT,0
1,BEAM,0
2,OG,0
3,BTC,0
4,WPR,0


In [92]:
res = requests.get("https://oauth.reddit.com/r/cryptocurrency/hot", headers=headers, params={'limit': 10})

for post in res.json()['data']['children']:
    if 'Daily Discussion' in post['data']['title']:
        daily_id = post['data']['id']
        break

In [102]:
res2 = requests.get("https://oauth.reddit.com/r/cryptocurrency/comments/" + daily_id, headers=headers,
                    params={'limit': 1000})
len(res2.json()[1]['data']['children'])

166

In [103]:
def get_comments(p_comment):
    if 'body' not in p_comment:
        return ''
    text = p_comment['body']
    replies = p_comment['replies']
    if len(replies) > 0:
        for c_comment in replies['data']['children']:
            text += ' ' + get_comments(c_comment['data'])
    return text

for post in res2.json()[1]['data']['children']:
    text = get_comments(post['data'])
    for coin in coins:
        if coin in text:
            df.loc[df['coin'] == coin, 'mentions'] += 1

df = df.sort_values(by='mentions', ascending=False)
df.head(10)

,coin,mentions
3,BTC,31
17,ETH,16
391,LTO,12
80,OM,10
354,ICX,10
226,VET,9
254,XLM,7
150,ONE,6
228,ADA,6
36,ZIL,5


In [ ]:
res = requests.get("https://oauth.reddit.com/r/cryptocurrency/new", headers=headers, params={'limit': 50})

for post in res.json()['data']['children']:
    text = post['data']['title'] + ' ' + post['data']['selftext']
    res2 = requests.get("https://oauth.reddit.com/r/cryptocurrency/comments/" + post['data']['id'],
                        headers=headers)
    for p_comment in res2.json():
        for c_comment in p_comment['data']['children']:
            if 'body' in c_comment['data'].keys():
                text = text + ' ' + c_comment['data']['body']
    for coin in coins:
        if coin in text:
            df.loc[df['coin'] == coin, 'mentions'] += 1

df = df.sort_values(by='mentions', ascending=False)
df.head(10)

In [ ]:
res = requests.get("https://oauth.reddit.com/r/cryptocurrency/hot", headers=headers, params={'limit': 100})

for post in res.json()['data']['children']:
    text = post['data']['title'] + ' ' + post['data']['selftext']
    for coin in coins:
        if coin in text:
            df.loc[df['coin'] == coin, 'mentions'] += 1

df = df.sort_values(by='mentions', ascending=False)
df.head(10)

In [ ]:
df.head(10).to_dict('records')

In [ ]:
def get_reddit_trending(coins, sub='cryptocurrency', mode='hot'):
    auth = requests.auth.HTTPBasicAuth(secrets['REDDIT_API_KEY'], secrets['REDDIT_SECRET_KEY'])
    headers = {'User-Agent': 'MyBot/0.0.1'}
    data = {'grant_type': 'password', 'username': secrets['REDDIT_USERNAME'], 'password': secrets['REDDIT_PASSWORD']}
    res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
    TOKEN = res.json()['access_token']
    headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
    df = pd.DataFrame(columns=['coin', 'mentions'])
    df['coin'] = coins
    df['mentions'] = 0
    res = requests.get('https://oauth.reddit.com/r/{}/{}'.format(sub, mode), headers=headers, params={'limit': 100})
    for post in res.json()['data']['children']:
        text = post['data']['title'] + ' ' + post['data']['selftext']
        for coin in coins:
            if coin in text:
                df.loc[df['coin'] == coin, 'mentions'] += 1
    df = df.sort_values(by='mentions', ascending=False)
    return df.head(10).to_dict('records')

In [ ]:
get_reddit_trending(coins)